In [3]:
import torch
import numpy as np



In [4]:
ts = torch.tensor([[1,2],[3,4]])
print(ts)
print(ts.shape)
print(ts.dtype)
print(ts.device)

tensor([[1, 2],
        [3, 4]])
torch.Size([2, 2])
torch.int64
cpu


In [5]:
ts2 = torch.tensor([[1,2]])
print(torch.cat([ts,ts2],dim=0))
print(ts.matmul(ts2.T))
print(ts.mul(ts2))
print(ts2.T)

tensor([[1, 2],
        [3, 4],
        [1, 2]])
tensor([[ 5],
        [11]])
tensor([[1, 4],
        [3, 8]])
tensor([[1],
        [2]])


In [6]:
print(ts2.matmul(ts.matmul(ts2.T)).item())
print(ts.add(10))



27
tensor([[11, 12],
        [13, 14]])


In [7]:
x = torch.tensor([1], requires_grad=True, dtype=torch.float)
y = torch.tensor([2], requires_grad=True, dtype=torch.float)

z = x**2 + y**3
z.backward()
print(x.grad)
print(y.grad)

tensor([2.])
tensor([12.])


In [8]:
import math
from numpy import ndarray

#FUNCTIONS

#OPERATORS

def annihilation(dimension: int) -> ndarray:
    """
    Returns a dense matrix of size dimension x dimension representing the annihilation
    operator in number basis.
    """
    offdiag_elements = np.sqrt(range(1, dimension))
    return np.diagflat(offdiag_elements, 1)

def creation(dimension: int) -> ndarray:
    """
    Returns a dense matrix of size dimension x dimension representing the creation
    operator in number basis.
    """
    return annihilation(dimension).T




In [9]:
#MORE NOISE RELATED FUNCTIONS

#T1 CAPACITIVE
import scipy as sp
import scipy.constants

def q_cap_fun(omega) -> torch.Tensor:
    return (
        1e6
        * (2 * np.pi * 6e9 / torch.abs(omega)) ** 0.7
    )

def calc_therm_ratio(
    omega: float, T: float
) -> torch.Tensor:
    return (sp.constants.hbar * omega) / (sp.constants.k * T)

def spectral_density_cap(omega,EJ,EC,EL,flux):
    T = 0.015
    therm_ratio = calc_therm_ratio(omega, T)
    s = (
        2
        * 8
        * EC
        / q_cap_fun(omega)
        * (1 / torch.tanh(0.5 * torch.abs(therm_ratio)))
        / (1 + torch.exp(-therm_ratio))
    )
    s *= (
        2 * np.pi
    )  # We assume that system energies are given in units of frequency
    return s



In [10]:
#MORE NOISE RELATED FUNCTIONS

#T1 CHARGE IMPEDANCE

def spectral_density_ci(omega,EJ,EC,EL,flux):
    # Note, our definition of Q_c is different from Zhang et al (2020) by a
    # factor of 2
    R_k = 50
    T = 0.015

    Q_c = R_k / (8 * np.pi * complex(R_k).real)
    therm_ratio = calc_therm_ratio(omega, T)
    s = (
        2
        * omega
        / Q_c
        * (1 / torch.tanh(0.5 * therm_ratio))
        / (1 + torch.exp(-therm_ratio))
    )
    return s


def spectral_density_fbl(omega,EJ,EC,EL,flux):
    """
    Our definitions assume that the noise_op is dH/dflux.
    """

    R_k = 50
    T = 0.015
    M = 400

    therm_ratio = calc_therm_ratio(omega, T)

    s = (
        2
        * (2 * np.pi) ** 2
        * M**2
        * omega
        * sp.constants.hbar
        / complex(R_k).real
        * (1 / torch.tanh(0.5 * therm_ratio))
        / (1 + torch.exp(-therm_ratio))
    )
    # We assume that system energies are given in units of frequency and that
    # the noise operator to be used with this `spectral_density` is dH/dflux.
    # Hence we have to convert  2 powers of frequency to standard units
    s *= 1e9 ** 2.0
    return s



In [11]:
def q_ind_fun(omega):
    T = 0.015

    therm_ratio = abs(calc_therm_ratio(omega, T))
    therm_ratio_500MHz = calc_therm_ratio(
        torch.tensor(2 * np.pi * 500e6), T
    )
    
    return (
        500e6
        * (
            torch.special.scaled_modified_bessel_k0(1 / 2 * therm_ratio_500MHz)
            * torch.sinh(1 / 2 * therm_ratio_500MHz)
            / torch.exp(1 / 2 * therm_ratio_500MHz)
        )
        / (
            torch.special.scaled_modified_bessel_k0(1 / 2 * therm_ratio)
            * torch.sinh(1 / 2 * therm_ratio)
            / torch.exp(1 / 2 * therm_ratio)
        )
    )

def spectral_density_ind(omega,EJ,EC,EL,flux):
    T = 0.015
    therm_ratio = calc_therm_ratio(omega,T)
    s = (
        2
        * EL
        / q_ind_fun(omega)
        * (1 / torch.tanh(0.5 * torch.abs(therm_ratio)))
        / (1 + torch.exp(-therm_ratio))
    )
    s *= (
        2 * np.pi
    )  # We assume that system energies are given in units of frequency
    return s

In [12]:
#GENERAL T1 function
from numpy import ndarray 
from typing import Any, Callable, Dict, List, Optional, Tuple, Union, cast

def t1(
        noise_op: torch.Tensor,
        spectral_density: Callable,
        eigvals: torch.Tensor,
        eigvecs: torch.Tensor,
        EJ: torch.Tensor,
        EC: torch.Tensor,
        EL: torch.Tensor,
        flux: torch.Tensor,
        i: int = 1,
        j: int = 0
    ):

        # We assume that the energies in `evals` are given in the units of frequency
        # and *not* angular frequency. The function `spectral_density` is assumed to
        # take as a parameter an angular frequency, hence we have to convert.


        ground = eigvecs[:,j]
        excited = eigvecs[:,i]
        ground_E = eigvals[j]
        excited_E = eigvals[i]

        omega = 2 * np.pi * (excited_E - ground_E) * 1e9

        s = (
            spectral_density(omega,EJ,EC,EL,flux) + spectral_density(-omega,EJ,EC,EL,flux)
        )
        
        rate = torch.matmul(noise_op,ground.unsqueeze(0).T)
        rate = torch.matmul(excited.unsqueeze(0).conj(),rate)
        rate = torch.abs(rate) **2 * s
        #print(torch.abs(torch.matmul(excited.unsqueeze(0).conj(),torch.matmul(noise_op,ground.unsqueeze(0).T))))

        return rate

In [13]:
#TENSORS

# PART 0 - INPUT

EJ = torch.tensor([8.9], requires_grad=True, dtype=torch.double)
EC = torch.tensor([2.5], requires_grad=True, dtype=torch.double)
EL = torch.tensor([0.5], requires_grad=True, dtype=torch.double)
flux = torch.tensor([0.5], requires_grad=True, dtype=torch.double)

dim = 110

# PART 1 - GET HAMILTONIAN

plasma_energy = math.sqrt(8.0*EL*EC)
diag_elements = [(i + 0.5) for i in range(dim)]
lc_osc = torch.tensor(np.diag(diag_elements),dtype=torch.double)
lc_osc = lc_osc * plasma_energy

phi_osc = (8.0 * EC / EL) ** 0.25
phi = (
        (torch.tensor(creation(dim),dtype=torch.double) + torch.tensor(annihilation(dim),dtype=torch.double))
        * phi_osc/ math.sqrt(2)
    )

argument = phi + 2 * np.pi * flux * torch.tensor(np.eye(dim),dtype=torch.double)

cos_phi = (torch.linalg.matrix_exp(argument*1j)+torch.linalg.matrix_exp(argument*-1j))/2
sin_phi = (torch.linalg.matrix_exp(argument*1j)-torch.linalg.matrix_exp(argument*-1j))/2j

n_op = (
            1j
            * (torch.tensor(creation(dim),dtype=torch.double) - torch.tensor(annihilation(dim),dtype=torch.double))
            / (phi_osc * math.sqrt(2))
        )

d_ham_d_flux = -2 * np.pi * EJ * sin_phi

H = lc_osc - EJ*cos_phi
#H = torch.triu(H, diagonal=1).T + torch.triu(H)
#H.to(torch.double)

#PART 2 - Get eigenvalues

eigvals,eigvecs = torch.linalg.eigh(H,UPLO='U')

# PART 3 - Find rate

t1_rate = torch.zeros([1,1],dtype=torch.double)

#3.1 t1_capacitive

#t1_rate += t1(n_op, spectral_density_cap, eigvals, eigvecs, EJ, EC, EL, flux)

#3.2 t1_charge_impedance

#t1_rate += t1(n_op, spectral_density_ci, eigvals, eigvecs, EJ, EC, EL, flux)

#3.3 t1_flux_bias_line

#t1_rate += t1(d_ham_d_flux, spectral_density_fbl, eigvals, eigvecs, EJ, EC, EL, flux)

#3.4 t1_inductive

t1_rate += t1(phi.to(torch.cdouble), spectral_density_ind, eigvals, eigvecs, EJ, EC, EL, flux)

#3.5 t1_quasiparticle_tunneling



# PART 4 - Get time

T1 = 1/t1_rate

#and now autograd

T1.backward()
print(T1)
EC.grad

#get to boundary, ignore component pointing out



tensor([[6450370.6882]], dtype=torch.float64, grad_fn=<MulBackward0>)


tensor([-890552.5026], dtype=torch.float64)

In [147]:
import scqubits as scq
import scqubits.core.noise as noise
import scqubits.core.units as units
import numpy as np
import torch
from numpy import ndarray
import math


fluxonium = scq.Fluxonium(EJ = 8.9,
                               EC = 2.5,
                               EL = 0.5,
                               flux = 0.5,
                               cutoff = 110)

fluxonium.t1_inductive(get_rate=False)

5489540.611565284